In [ ]:
import pandas as pd
import glob
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
from experiment_utils import Experiment, ExperimentFilter, Plotting

In [ ]:
experiments = Experiment.get_experiments()
experiments = list(filter(ExperimentFilter.by_experiment_name('rapl-update-intervals'), experiments))
experiment = ExperimentFilter.get_latest(experiments)
experiment

In [ ]:
data = pd.DataFrame()

for file in glob.glob(f'{experiment.path}/results_*.csv'):
    stem_parts = Path(file).stem.split('_')
    frequency = int(stem_parts[1])
    filter = stem_parts[2]
    try:
        c_state = stem_parts[3]
    except:
        c_state = ''

    partial = pd.read_csv(file, sep=',', index_col=False)
    partial['frequency_khz'] = frequency
    partial['filter'] = filter
    partial['c_state'] = c_state

    data = pd.concat([
        data,
        partial
    ], ignore_index=True)

data

In [ ]:
data['register'].unique()

In [ ]:
for register, group in data.groupby('register'):
    # print(register, group)
    valids = len(group[group['current_value'] != 0])
    invalids = len(group[group['current_value'] == 0])
    print(f'Register {hex(register)} contains {valids} valids and {invalids} invalids')

In [ ]:
valid_data = data[(data['register'] == 0x611) | (data['register'] == 0x612) | (data['register'] == 0x619)]
valid_data

In [ ]:
def get_data(df: pd.DataFrame, frequency_khz: int) -> pd.DataFrame:
    filtered_data = df
    filtered_data = filtered_data[filtered_data['frequency_khz'] == frequency_khz ]

    return filtered_data

In [ ]:
def get_register(df, register):
    return df[df['register'] == register]

def remove_nonupdated_values(df):
    return df[(df['current_value'] - df['previous_value']) != 0]

In [ ]:
ENERGY_DIFF_UJ = 'Energy difference since last update [µJ]'
ENERGY_DIFF_MJ = 'Energy difference since last update [mJ]'
TIME_DIFF_REGISTER_MS = 'Time since last update from register [ms]'
TIME_DIFF_MS = 'Time since last update from rdtsc [ms]'

hue_order=['NOFILTER ', 'FILTER ', 'NOFILTER POLL', 'FILTER POLL']

In [ ]:
def plot_package_energy(filtered_data: pd.DataFrame, frequency_khz: int):
    register_data = remove_nonupdated_values(get_register(filtered_data, 0x612))
    register_data['timestamp_diff'] = register_data['current_timestamp'].diff() % (2**32)
    register_data['timestamp_diff_rapl'] = register_data['current_timestamp_rapl'].diff() % (2**32)
    register_data['value_diff'] = register_data['current_value'].diff()
    register_data[ENERGY_DIFF_UJ] = register_data['value_diff'] * ((1/2) ** 0xe)
    MSR_PACKAGE_ENERGY_TIME_STATUS_HUE = 'MSR_PACKAGE_ENERGY_TIME_STATUS'
    register_data[MSR_PACKAGE_ENERGY_TIME_STATUS_HUE] = register_data['filter'] + ' ' + register_data['c_state']
    print(f'Unique keys are {register_data[MSR_PACKAGE_ENERGY_TIME_STATUS_HUE].unique()}')
    for key, values in register_data.groupby(MSR_PACKAGE_ENERGY_TIME_STATUS_HUE):
        print(key, len(values))
    # Unit is 10ns
    register_data[TIME_DIFF_REGISTER_MS] = register_data['timestamp_diff_rapl'] * 10 / 1e6
    register_data[TIME_DIFF_MS] = register_data['timestamp_diff'] / 2E6

    register_data = register_data[register_data[TIME_DIFF_REGISTER_MS] > 0]
    register_data = register_data[register_data[TIME_DIFF_REGISTER_MS] <= 10]
    register_data = register_data[register_data[ENERGY_DIFF_UJ] > 0]
    register_data = register_data[register_data[ENERGY_DIFF_UJ] < 10]

    register_data = register_data.reset_index()


    plt.rcParams['figure.figsize'] = (11, 6)
    fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(nrows=2,
                                                        ncols=3,
                                                        sharex=False,
                                                        sharey=False,
                                                        width_ratios=[5, 5, 1],
                                                        height_ratios=[1, 5])

    sns.kdeplot(ax=ax5,
                    data=register_data,
                    x=TIME_DIFF_REGISTER_MS,
                    y=ENERGY_DIFF_UJ,
                    hue=MSR_PACKAGE_ENERGY_TIME_STATUS_HUE,
                    hue_order=hue_order,
                    common_norm=False)
    ax5.set_xlim(0, 5)
    ax5.set_ylim(ymin=0)
    ax5.set(ylabel=None)
    ax5.get_yaxis().set_ticklabels([])


    sns.kdeplot(ax=ax6, data=register_data, y=ENERGY_DIFF_UJ, hue=MSR_PACKAGE_ENERGY_TIME_STATUS_HUE,
                    hue_order=hue_order)


    sns.kdeplot(ax=ax4,
                    data=register_data,
                    x=TIME_DIFF_MS,
                    y=ENERGY_DIFF_UJ,
                    hue=MSR_PACKAGE_ENERGY_TIME_STATUS_HUE,
                    hue_order=hue_order,
                    common_norm=False)
    ax4.set_xlim(0, 5)
    ax4.set_ylim(ymin=0)


    sns.kdeplot(ax=ax1, data=register_data, x=TIME_DIFF_MS, hue=MSR_PACKAGE_ENERGY_TIME_STATUS_HUE,
                    hue_order=hue_order)
    sns.kdeplot(ax=ax2, data=register_data, x=TIME_DIFF_REGISTER_MS, hue=MSR_PACKAGE_ENERGY_TIME_STATUS_HUE,
                    hue_order=hue_order)

    ax1.get_legend().remove()
    ax2.get_legend().remove()

    ax1.spines["top"].set_visible(False)
    ax1.spines["left"].set_visible(False)
    ax1.spines["right"].set_visible(False)
    # ax1.tick_params(left=False, bottom=False)
    ax1.get_yaxis().set_visible(False)
    ax1.get_xaxis().set_ticklabels([])
    ax1.set(xlabel=None)
    ax1.set_xlim(0, 5)

    ax2.spines["top"].set_visible(False)
    ax2.spines["left"].set_visible(False)
    ax2.spines["right"].set_visible(False)
    # ax2.tick_params(left=False, bottom=False)
    ax2.get_yaxis().set_visible(False)
    ax2.get_xaxis().set_ticklabels([])
    ax2.set(xlabel=None)
    ax2.set_xlim(0, 5)

    ax4.get_legend().remove()
    ax6.get_legend().remove()

    ax3.spines["top"].set_visible(False)
    ax3.spines["left"].set_visible(False)
    ax3.spines["right"].set_visible(False)
    ax3.spines["bottom"].set_visible(False)
    ax3.tick_params(left=False, bottom=False)
    ax3.get_xaxis().set_ticklabels([])
    ax3.get_yaxis().set_ticklabels([])

    ax6.spines["top"].set_visible(False)
    ax6.spines["bottom"].set_visible(False)
    ax6.spines["right"].set_visible(False)
    ax6.get_xaxis().set_visible(False)
    ax6.set(ylabel=None)
    ax6.get_yaxis().set_ticklabels([])
    ax6.set_ylim(ax5.get_ylim())

    fig.tight_layout()

    Plotting.savefig(experiment, f'{MSR_PACKAGE_ENERGY_TIME_STATUS_HUE}_{frequency_khz}.pdf', annotations_y_offset=0, annotations_y_spacing=0.025)

    plt.show()

In [ ]:
def plot_ram_energy(filtered_data: pd.DataFrame, frequency_khz: int):
    register_data = remove_nonupdated_values(get_register(filtered_data, 0x619))
    register_data['timestamp_diff'] = register_data['current_timestamp'].diff() % (2**32)
    register_data['value_diff'] = register_data['current_value'].diff() % (2**32)
    register_data[ENERGY_DIFF_MJ] = register_data['value_diff'] * 61 / 1000

    MSR_RAM_ENERGY_HUE = 'MSR_RAM_ENERGY'
    register_data[MSR_RAM_ENERGY_HUE] = register_data['filter'] + ' ' + register_data['c_state']
    print(f'Unique keys are {register_data[MSR_RAM_ENERGY_HUE].unique()}')

    for key, values in register_data.groupby(MSR_RAM_ENERGY_HUE):
        print(key, len(values))
    # Unit is 10ns
    register_data[TIME_DIFF_MS] = register_data['timestamp_diff'] / 2E6

    register_data = register_data[register_data[TIME_DIFF_MS] > 0]
    register_data = register_data[register_data[TIME_DIFF_MS] <= 3]
    register_data = register_data[register_data[ENERGY_DIFF_MJ] > 0]
    register_data = register_data[register_data[ENERGY_DIFF_MJ] < 50]

    register_data = register_data.reset_index()


    plt.rcParams['figure.figsize'] = (6, 6)
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2,
                                                ncols=2,
                                                sharex=False,
                                                sharey=False,
                                                width_ratios=[5, 1],
                                                height_ratios=[1, 5])

    sns.kdeplot(ax=ax4, data=register_data, y=ENERGY_DIFF_MJ, hue=MSR_RAM_ENERGY_HUE,
                    hue_order=hue_order)
    ax4.get_legend().remove()
    ax4.spines["top"].set_visible(False)
    ax4.spines["bottom"].set_visible(False)
    ax4.spines["right"].set_visible(False)
    ax4.get_xaxis().set_visible(False)
    ax4.set(ylabel=None)
    ax4.get_yaxis().set_ticklabels([])

    sns.kdeplot(ax=ax3,
                    data=register_data,
                    x=TIME_DIFF_MS,
                    y=ENERGY_DIFF_MJ,
                    hue=MSR_RAM_ENERGY_HUE,
                    hue_order=hue_order,
                    common_norm=False)
    ax3.set_xlim(xmin=0, xmax=3)
    ax3.set_ylim(ymin=0)

    sns.kdeplot(ax=ax1, data=register_data, x=TIME_DIFF_MS, hue=MSR_RAM_ENERGY_HUE,
                    hue_order=hue_order)
    ax1.get_legend().remove()
    ax1.spines["top"].set_visible(False)
    ax1.spines["left"].set_visible(False)
    ax1.spines["right"].set_visible(False)
    ax1.get_yaxis().set_visible(False)
    ax1.set(xlabel=None)
    ax1.get_xaxis().set_ticklabels([])
    ax1.set_xlim(xmin=0, xmax=3)

    ax2.spines["top"].set_visible(False)
    ax2.spines["left"].set_visible(False)
    ax2.spines["right"].set_visible(False)
    ax2.spines["bottom"].set_visible(False)
    ax2.tick_params(left=False, bottom=False)
    ax2.get_xaxis().set_ticklabels([])
    ax2.get_yaxis().set_ticklabels([])


    ax4.set_ylim(ax3.get_ylim())

    fig.tight_layout()

    Plotting.savefig(experiment, f'{MSR_RAM_ENERGY_HUE}_{frequency_khz}.pdf', annotations_y_offset=0, annotations_y_spacing=0.025)

    plt.show()

In [ ]:
def plot_for_frequency(frequency_khz: int):
    print(f'Plotting for frequency {frequency_khz}kHz')
    filtered_data = valid_data[valid_data['frequency_khz'] == frequency_khz]
    print(filtered_data['register'].unique())
    plot_package_energy(filtered_data=filtered_data, frequency_khz=frequency_khz)
    plot_ram_energy(filtered_data=filtered_data, frequency_khz=frequency_khz)

In [ ]:
for frequency_khz in valid_data['frequency_khz'].unique():
    plot_for_frequency(frequency_khz=frequency_khz)